# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [17]:
from azureml.core import Workspace, Experiment
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.core.model import Model, InferenceConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice, loguniform
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.webservice import AciWebservice
import sklearn
import os
import joblib
import pandas as pd

## Dataset

Data accessed and cleaned in the train.py script

In [18]:
ws = Workspace.from_config()
experiment_name = 'hyperdrive'

experiment=Experiment(ws, experiment_name)

run = experiment.start_logging()

## Start compute cluster

In [3]:
cpu_cluster_name = "ML-Cluster2"
compute_config = AmlCompute.provisioning_configuration("Standard_D2_V2", min_nodes=0, max_nodes=2)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)

InProgress...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [19]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.05, evaluation_interval=5, delay_evaluation=10)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    "--C": loguniform(-10, 10),
    "--max_iter": choice(50,100,200,400)
})

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn("./", compute_target=cpu_cluster, entry_script="./source_dir/train.py")

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling, 
                                    estimator=estimator, 
                                    policy=early_termination_policy,
                                    primary_metric_name="Accuracy", 
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,  
                                    max_total_runs=20)

In [20]:
#TODO: Submit your experiment
run = experiment.submit(hyperdrive_run_config, show_output=True)
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [31]:
best_run = run.get_best_run_by_primary_metric()
best_run_metric = best_run.get_metrics()

print(best_run)
print(best_run_metric)

best_run.get_file_names()

if "outputs" not in os.listdir():
    os.mkdir("./outputs")


best_model=best_run.register_model(
    model_name='hdr', 
    model_path='outputs/model.joblib',
    tags=best_run_metric, 
    model_framework=Model.Framework.SCIKITLEARN, 
    model_framework_version=sklearn.__version__)

# Save the best model
#best_run.download_file(best_run.get_file_names()[-1], output_file_path='./outputs/')

Run(Experiment: hyperdrive,
Id: HD_0c342d4b-7e90-482d-b9ba-88b4179bd5ac_12,
Type: azureml.scriptrun,
Status: Completed)
{'Regularization Strength:': 60.87279320432503, 'Max iterations:': 100, 'Accuracy': 0.6763636363636364}


In [22]:
metrics = pd.DataFrame.from_dict(best_run_metric, orient='index', columns=['metric value'])
metrics

,metric value
Regularization Strength:,60.872793
Max iterations:,100.000000
Accuracy,0.676364


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [32]:
# Define inference config

env = best_run.get_environment()
inference_config = InferenceConfig(
    environment= env,
    source_directory = "./source_dir",
    entry_script="score.py"
)

In [33]:
# Define a deployment configuration on the Webservice

deploy_config = AciWebservice.deploy_configuration(
    cpu_cores=0.5, memory_gb=1, auth_enabled=True, enable_app_insights=True
)

In [34]:
# Deploy the model

service = Model.deploy(
    workspace = ws,
    name = "sinter-scrap-model2",
    models = [best_model],
    inference_config = inference_config,
    deployment_config = deploy_config,
    overwrite = True
)

service.wait_for_deployment(show_output=True)
print(service.get_logs())

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-10-21 21:52:01+00:00 Creating Container Registry if not exists.
2021-10-21 21:52:01+00:00 Registering the environment.
2021-10-21 21:52:03+00:00 Use the existing image.
2021-10-21 21:52:03+00:00 Generating deployment configuration.
2021-10-21 21:52:05+00:00 Submitting deployment to compute.
2021-10-21 21:52:07+00:00 Checking the status of deployment sinter-scrap-model2..
2021-10-21 21:57:40+00:00 Checking the status of inference endpoint sinter-scrap-model2.
Succeeded
ACI service creation operation finished, operation "Succeeded"
2021-10-21T21:56:46,738474300+00:00 - iot-server/run 
2021-10-21T21:56:46,742455400+00:00 - rsyslog/run 
2021-10-21T21:56:46,770175300+00:00 - nginx/run 
2021-10-21T21:56:46,765985700+00:00 - gunicorn/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. E

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# Use the first 10 rows of the actual cleaned data as a test data
test_data = clean_data[0:10]

# Drop the column that will be predicted
test_data = test_data.drop("HAS SCRAP AT SINTER", axis=1)

# Convert pandas DataFrame into the dictionary
test_dict = ({'data':test_data.to_dict(orient='records')})

# Encode data into the JSON object
test_sample = json.dumps(test_dict)


# Pass the JSON object into our deployed models endpoint
# and print the predicted results
print(service.run(test_sample))

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

